# Stackleberg equilibrium

This notebook is used to find the stackleberg equilibrium for a game in extensive form translated into normal-form.

In [443]:
from amplpy import AMPL
import pandas as pd

In [444]:
def data_preprocessing(players, actions, utility_matrices):
    actions_encoded = {player: [i for i in range(len(actions[player]))] for player in players}
    utilities_df = {player: pd.DataFrame(utility_matrices[player], index=actions_encoded['leader'], columns=actions_encoded['follower']) for player in players}
    
    return utilities_df, actions_encoded

In [445]:
def stackleberg_equilibrium(players, actions, utilities):
    utilities_df, actions_encoded = data_preprocessing(players, actions, utilities)
    
    ampl = AMPL()
    ampl.read('normal_stackleberg.mod')
    
    ampl.set['Al'] = actions_encoded['leader']
    ampl.set['Af'] = actions_encoded['follower']
    
    ampl.param['Ul'] = utilities_df['leader']
    ampl.param['Uf'] = utilities_df['follower']
    
    optimal = {}
    for action_follower in actions_encoded['follower']:
        ampl.param['Af_bar'] = action_follower
        ampl.solve(solver='gurobi')
        assert ampl.solve_result == "solved"
        
        obj = ampl.getObjective('obj').value()
        sl = ampl.getVariable('sl').getValues().toPandas()
        
        if obj > optimal.get('obj', -float('inf')):
            optimal = {'Af_bar': actions['follower'][action_follower], 'obj': obj, 'sl': sl}
    
    ampl.close()
    
    return optimal
    

In [446]:
# Game
players = ['leader', 'follower']

A_1 = ["t1", "t2", "t3", "t4"]
A_2 = ["t1", "t2", "t3", "t4"]

I = {'leader': A_1, 'follower': A_2}

U1 = [[1, 0.5, 0.5, 0.5],
    [0.5, 1, 0.5, 0.5],
    [0.5, 0.5, 1, 0.5],
    [0.5, 0.5, 0.5, 1]]

U2 = [[0, 0.4, 0.3, 0.3],
    [0.6, 0, 0.4, 0.3],
    [0.7, 0.6, 0, 0.4],
    [0.7, 0.7, 0.6, 0]]

U = {'leader': U1, 'follower': U2}

optimal = stackleberg_equilibrium(players, I, U)

print("\nRESULTS:")
print("Optimal action for follower: ", optimal['Af_bar'])
print("Optimal strategy for leader: ")
for index, row in optimal['sl'].iterrows():
    print(f" {I['leader'][index]} -> {row['sl.val']}")
print("Optimal objective value: ", optimal['obj'])

Gurobi 9.5.1: optimal solution; objective 0.7598039216
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.662745098
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.562745098
Gurobi 9.5.1: optimal solution; objective 0.5147058824

RESULTS:
Optimal action for follower:  t1
Optimal strategy for leader: 
 t1 -> 0.5196078431372549
 t2 -> 0.3254901960784313
 t3 -> 0.1254901960784314
 t4 -> 0.02941176470588236
Optimal objective value:  0.7598039215686274


# Bayesian game

In [447]:
def bayesian_data_preprocesing(players, actions, utility_matrices, types_follower, probabilities_follower):
    actions_encoded = {player: [i for i in range(len(actions[player]))] for player in players}
    types_follower_encoded = [i for i in range(len(types_follower))]
    probabilities_follower_df = pd.DataFrame(probabilities_follower.values(), index=types_follower_encoded)
    utilities_df = {
        'leader': pd.DataFrame(utility_matrices['leader'], index=actions_encoded['leader'], columns=actions_encoded['follower']),
        'follower': [pd.DataFrame(utility_matrices['follower'][type_follower], index=actions_encoded['leader'], columns=actions_encoded['follower']) for type_follower in types_follower]
    }
    
    utilities_df['follower'] = pd.concat(utilities_df['follower'], keys=types_follower_encoded)
    
    return utilities_df, actions_encoded, types_follower_encoded, probabilities_follower_df

In [448]:
def bayesian_stackleberg_equilibrium(players, actions, utility_matrices, types_follower, probabilities_follower):
    utilities_df, actions_encoded, types_follower_encoded, probabilities_follower_df = bayesian_data_preprocesing(players, actions, utility_matrices, types_follower, probabilities_follower)
    
    ampl = AMPL()
    ampl.read('bayesian_normal_stackleberg.mod')
    
    ampl.set['Al'] = actions_encoded['leader']
    ampl.set['Af'] = actions_encoded['follower']
    ampl.set['T'] = types_follower_encoded

    ampl.param['P'] = probabilities_follower_df
    ampl.param['Ul'] = utilities_df['leader']
    ampl.param['Uf'] = utilities_df['follower']
    
    
    optimal = {}
    for action_follower_i in actions_encoded['follower']:
        for action_follower_j in range(action_follower_i, len(actions_encoded['follower'])):
            action_follower_df = pd.DataFrame([action_follower_i, action_follower_j], index=types_follower_encoded)
            ampl.param['Af_bar'] = action_follower_df
        
            ampl.solve(solver='gurobi')
            assert ampl.solve_result == "solved"
            
            obj = ampl.getObjective('obj').value()
            sl = ampl.getVariable('sl').getValues().toPandas()
            
            if obj > optimal.get('obj', -float('inf')):
                optimal = {'Af_bar': action_follower_df, 'obj': obj, 'sl': sl}
            
    ampl.close()
    
    return optimal

In [449]:
# Game
players = ['leader', 'follower']

A_1 = ["t1", "t2", "t3", "t4"]
A_2 = ["t1", "t2", "t3", "t4"]

I = {'leader': A_1, 'follower': A_2}

T = ['type1', 'type2'] # Types of the follower

P = {'type1': 0.5, 'type2': 0.5} # Probability of each type


Ul = [[1, 0.5, 0.5, 0.5],
    [0.5, 1, 0.5, 0.5],
    [0.5, 0.5, 1, 0.5],
    [0.5, 0.5, 0.5, 1]]

Uf_type1 = [[0, 0.4, 0.3, 0.3],
            [0.6, 0, 0.4, 0.3],
            [0.7, 0.6, 0, 0.4],
            [0.7, 0.7, 0.6, 0]]

Uf_type2 = [[0, 0.7, 0.6, 0.6],
            [0.4, 0, 0.7, 0.6],
            [0.3, 0.4, 0, 0.7],
            [0.3, 0.3, 0.4, 0]]

Uf = {'type1': Uf_type1, 'type2': Uf_type2}

U = {'leader': Ul, 'follower': Uf}

optimal = bayesian_stackleberg_equilibrium(players, I, U, T, P)

print("\nRESULTS:")
print("Optimal action for follower: ")
for index, type_follower in enumerate(T):
    print(f"{type_follower} -> {I['follower'][optimal['Af_bar'].loc[index].values[0]]}")
print("Optimal strategy for leader: ")
for index, row in optimal['sl'].iterrows():
    print(f" {I['leader'][index]} -> {row['sl.val']}")
print("Optimal objective value: ", optimal['obj'])

Gurobi 9.5.1: optimal solution; objective 0.5085066163
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.5884502924
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.6470238095
2 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.6593567251
2 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.5660377358
3 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.6023809524
5 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.6293859649
2 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.5523809524
4 simplex iterations
Gurobi 9.5.1: optimal solution; objective 0.5529761905
Gurobi 9.5.1: optimal solution; objective 0.5147058824
1 simplex iterations

RESULTS:
Optimal action for follower: 
type1 -> t1
type2 -> t4
Optimal strategy for leader: 
 t1 -> 0.2368421052631579
 t2 -> 0.11695906432748537
 t3 -> 0.2456140350877193
 t4 -> 0.40058479532163743
Optimal objective value:  0.6593567251461988
